In [1]:
import numpy as np
## 42类标注
state_set=['nz', 'Bg', 'nr', 'h', 'Mg', 's', 'nt', 'Ng', 'k', 'p', 'e', 'Rg', 'r', 'y', 't', 'q', 'n', 'ns', 'Vg', 'm', 'vd', 'nx', 'vn', 'i', 'z', 'u', 'b', 'w', 'Ag', 'j', 'c', 'l', 'v', 'd', 'a', 'f', 'o', 'ad', 'Tg', 'an', 'Yg', 'Dg']
trans = {}
emit = {}
start = {}
count_dic = {}  #每个状态在训练集中出现的次数
line_num = 0    #训练集语句数量
word_set = set()    #训练数据集中所有字的集合

In [2]:
#初始化所有概率矩阵
def Init_Array():
    for state0 in state_set:
        trans[state0] = {}
        for state1 in state_set:
            trans[state0][state1] = 0.0
    for state in state_set:
        start[state] = 0.0
        emit[state] = {}
        emit[state]['unknown']=0;
        count_dic[state] = 0

In [3]:
#将参数估计的概率取对数，对概率0取无穷小-3.14e+100
def Prob_Array():
    for key in start:
        if start[key] == 0:
            start[key] = -3.14e+100
        else:
            start[key] = np.log(start[key] / line_num)
    for key0 in trans:
        for key1 in trans[key0]:
#             if trans[key0][key1] == 0.0:
#                 trans[key0][key1] = -3.14e+100
#             else:
            trans[key0][key1] = np.log(1+trans[key0][key1] / count_dic[key0]+len(trans[key0]))
    # print(trans)
    for key in emit:
        for word in emit[key]:
#             if emit[key][word] == 0.0:
#                 emit[key][word] = -3.14e+100
#             else:
            #add one smoothing
            emit[key][word] = np.log((emit[key][word]+1) /(count_dic[key]+len(emit[key])))

                

In [4]:

#Viterbi算法求测试集最优状态序列
def Viterbi(sentence,array_pi,array_a,array_b):
    tab = [{}]  #动态规划表
    path = {}

    for state in state_set:
        #print(array_pi)
        
        #发射矩阵中某状态不存在该首字符sentence[0],则概率降至最低
        if sentence[0] not in array_b[state]:
            array_b[state][sentence[0]] = -3.14e+100
        tab[0][state] = array_pi[state] + array_b[state][sentence[0]]
        path[state] = [state]
    for i in range(1,len(sentence)):
        tab.append({})
        new_path = {}
        for state0 in state_set:
            items = []
            for state1 in state_set:
                if sentence[i] not in array_b[state0]:  #所有在测试集出现但没有在训练集中出现的字符
                    prob = tab[i - 1][state1] + array_a[state1][state0] + array_b[state0]['unknown']
                else:
                    prob = tab[i-1][state1] + array_a[state1][state0] + array_b[state0][sentence[i]]    #计算每个字符对应STATES的概率
                items.append((prob,state1))
           
            best = max(items)   #best:(prob,state)
            # print(best)
            tab[i][state0] = best[0]
            # print(tab[i][state0])
            new_path[state0] = path[best[1]] + [state0]
        path = new_path

    prob, state = max([(tab[len(sentence) - 1][state], state) for state in state_set])
    return path[state]

In [5]:
def train(str):
    print("training ",str)
    global state_set
    global line_num
    trainset = open(str, encoding='utf-8')     #读取训练集1
    loop_num = 0
    for line in trainset:
        line = line.strip()
        if not len(line):
            continue
        line_num+=1
        
        global state_set
        word_list=[]
        state_list=[]
        
        ntflag = False
        stdline = ""
        templine = ""
        for k in range(len(line)):
            if not ntflag :
                if line[k] == '[':
                    ntflag = True
                else:
                    stdline += line[k]
            else :
                if line[k] == ']':
                    templist = templine.split()
                    templine =""
                    for item in templist:
                        item = item.split('/')
                        stdline+=item[0]
                    stdline+='/'
                    ntflag = False
                else:
                    templine+=line[k]
        # print(stdline)            
        stdlist = stdline.split()
        
        for i in range(1,len(stdlist)):
            item = stdlist[i].split('/')
            # print(item)
            word_list.append(item[0])
            state_list.append(item[1])
        
        start[state_list[0]] += 1
        
        
        for j in range(len(state_list)-1):
            trans[state_list[j]][state_list[j+1]] += 1  #trans计算状态转移概率

        for p in range(len(state_list)):
            count_dic[state_list[p]] += 1  # 记录每一个状态的出现次数
            for state in state_set:
                if word_list[p] not in emit[state]:
                    emit[state][word_list[p]] = 0.0  #保证每个字都在state_set的字典中
            emit[state_list[p]][word_list[p]] += 1  # emit用于计算发射概率

#     print('参数估计结果')
#     print('初始状态分布')
#     print(start)
#     print('状态转移矩阵')
#     print(trans)
#     print('发射矩阵')
#     print(emit)
    #    state_set = state_set | set(state_list)
        
    #     print(loop_num)
        loop_num +=1
    #     if(loop_num ==10): break;
    #print(loop_num)

In [6]:
def predict(str):
    print("start to predict",str)
    testset = open(str, encoding='utf-8')       #读取测试集
    word_num = 0
    correct_num = 0
    line_counter = 0
    for line in testset:
        line = line.strip()
        if len(line)==0:continue #跳过空行
        line_counter+=1
        if line_counter%50==0:
            print(line_counter)
        if line_counter==1000:
            break;
        ntflag = False
        stdline = ""
        templine = ""
        for k in range(len(line)):
            if not ntflag :
                if line[k] == '[':
                    ntflag = True
                else:
                    stdline += line[k]
            else :
                if line[k] == ']':
                    templist = templine.split()
                    templine =""
                    for item in templist:
                        item = item.split('/')
                        stdline+=item[0]
                    stdline+='/'
                    ntflag = False
                else:
                    templine+=line[k]
        # print(stdline)            
        stdlist = stdline.split()
        
        word_list=[]
        key_state_list=[]
        #test_state_list=[]
        
        for i in range(len(stdlist)):
            if i == 0:
                continue
            item = stdlist[i].split('/')
            # print(item)
            word_list.append(item[0])
            key_state_list.append(item[1])
        
        word_num += len(word_list)
        
        test_state_list = Viterbi(word_list, start, trans, emit)
        
        
        for i in range(len(key_state_list)):
            if(key_state_list[i]==test_state_list[i]):
                correct_num += 1
    precision = correct_num/word_num
    print("precision:",precision)

In [7]:
def crossvalid(filelist):
    for i in range(len(filelist)):
        print(i)
        Init_Array()
        for j in range(len(filelist)):
            if j!=i:
                train(filelist[j])
        Prob_Array()
        print(start)
        predict(filelist[i])

In [8]:
filelist = ['./data/199801.txt','./data/199802.txt','./data/199803.txt','./data/199804.txt']
crossvalid(filelist)

0
training  ./data/199802.txt
training  ./data/199803.txt
training  ./data/199804.txt
{'nz': -5.194494379010218, 'Bg': -3.14e+100, 'nr': -2.0699612720593965, 'h': -3.14e+100, 'Mg': -3.14e+100, 's': -6.236434280579417, 'nt': -2.6841605977678915, 'Ng': -7.140404528065531, 'k': -3.14e+100, 'p': -2.3229247772311683, 'e': -8.393167496560899, 'Rg': -3.14e+100, 'r': -2.2580665349701996, 'y': -11.032224826176158, 't': -2.6552136653597826, 'q': -8.729639733182111, 'n': -2.1462305110233473, 'ns': -2.7627240589955426, 'Vg': -7.203583429687063, 'm': -2.4995520639115334, 'vd': -7.813349001307957, 'nx': -7.535717264709677, 'vn': -4.749958079280151, 'i': -5.510763908313911, 'z': -6.989173558341608, 'u': -11.032224826176158, 'b': -5.124141888007227, 'w': -2.7252589608075835, 'Ag': -8.46727546871462, 'j': -4.002251914469771, 'c': -4.061494748032632, 'l': -4.898826783179509, 'v': -2.4083320161008643, 'd': -3.865958852042519, 'a': -4.3652680337469505, 'f': -5.938474625369395, 'o': -10.339077645616213, 'a

KeyboardInterrupt: 

In [20]:
# Prob_Array()
# predict('./data/199801.txt')